In [2]:
import pandas as pd

In [7]:
file = r"C:\Users\ruw\Desktop\RA\penicilin\data\cf.xlsx"
sheets = ["recipe", "ecotox"]

In [ ]:
recipe = pd.read_excel(io=file, sheet_name=sheets[0], index_col=0)
ecotox = pd.read_excel(io=file, sheet_name=sheets[1], index_col=0)


,freshwater (H),marine (H),terrestial (H)
"1,2-Dichlorobenzene",0.83600,0.141000,1.760000e+00
1-Pentanol,0.04110,0.001250,6.950000e-01
"2,4-D",2.62000,0.145000,5.400000e-04
2-Methyl-1-propanol,0.02480,0.001440,7.980000e-01
2-Propanol,0.00477,0.000299,1.880000e-02
...,...,...,...
Zinc II,211.00000,299.000000,1.290000e-14
m-Xylene,0.49200,0.008900,2.670000e-02
o-Xylene,0.52500,0.009910,6.200000e-02
t-Butyl methyl ether,0.00670,0.000414,3.400000e-02


In [23]:
scaling_factors = pd.DataFrame(None,index=recipe.index, columns=recipe.columns)

1,2-Dichlorobenzene       586.984454
1,4-Butanediol              7.309332
1-Pentanol                 15.568038
2-Methyl-1-propanol         8.538837
2-Propanol                  2.463417
                            ...     
Zinc II                 38584.584793
m-Xylene                  293.377640
o-Xylene                  220.767481
t-Butyl methyl ether        6.633173
t-Butylamine              125.389298
Name: ecotox, Length: 99, dtype: float64

In [24]:
for rcol in recipe.columns:
    for ridx, rrow in recipe.iterrows():
        for ecol in ecotox.columns:
            for eidx, erow in ecotox.iterrows():
                if ridx == eidx:
                    scaling_factors.at[ridx, rcol] = float(rrow[rcol])/float(erow[ecol])

scaling_factors

,freshwater (H),marine (H),terrestial (H)
"1,2-Dichlorobenzene",0.001424,0.00024,0.002998
1-Pentanol,0.00264,0.00008,0.044643
"2,4-D",NaN,NaN,NaN
2-Methyl-1-propanol,0.002904,0.000169,0.093455
2-Propanol,0.001936,0.000121,0.007632
...,...,...,...
Zinc II,0.005469,0.007749,0.0
m-Xylene,0.001677,0.00003,0.000091
o-Xylene,0.002378,0.000045,0.000281
t-Butyl methyl ether,0.00101,0.000062,0.005126


In [26]:
scaling_factors = scaling_factors.dropna()

In [27]:
file_name = r"C:\Users\ruw\Desktop\RA\penicilin\data\scaling_factors.xlsx"

with pd.ExcelWriter(file_name) as writer:
        scaling_factors.to_excel(writer, sheet_name="scaling factors", index=True, header=True)

In [1]:
from time import time

In [1]:
import bw2data as bd
bd.projects.set_current("Penicillin")

In [2]:
bd.databases

Databases dictionary with 6 object(s):
	biosphere3
	ev391consq
	ev391cutoff
	penicillin_consq
	penicillin_cut_off
	test

In [22]:
db = bd.Database("penicillin_cut_off")
our_activity = [act for act in db][0]
our_activity

'manufacturing of vial of penicillin' (unit, FR, None)

In [23]:
for exc in our_activity.exchanges():
    print(exc)

Exchange: 1 unit 'manufacturing of vial of penicillin' (unit, FR, None) to 'manufacturing of vial of penicillin' (unit, FR, None)>
Exchange: 0.006 kilogram 'market for aluminium alloy, AlLi' (kilogram, GLO, None) to 'manufacturing of vial of penicillin' (unit, FR, None)>
Exchange: 3.733333333333333e-05 kilowatt hour 'electricity production, hydro, pumped storage' (kilowatt hour, FR, None) to 'manufacturing of vial of penicillin' (unit, FR, None)>
Exchange: 0.00603125 kilogram 'market for extrusion, plastic pipes' (kilogram, GLO, None) to 'manufacturing of vial of penicillin' (unit, FR, None)>
Exchange: 0.00775 kilogram 'market for furniture, wooden' (kilogram, GLO, None) to 'manufacturing of vial of penicillin' (unit, FR, None)>
Exchange: 0.02634 kilogram 'market for glass tube, borosilicate' (kilogram, GLO, None) to 'manufacturing of vial of penicillin' (unit, FR, None)>
Exchange: 0.01065652 kilogram 'market for injection moulding' (kilogram, GLO, None) to 'manufacturing of vial of pe

In [5]:
db = bd.Database("penicillin_cut_off")
scenarios = ["sc1", "sc2", "sc3"] 
flow = []
    # Check if the database is case1
for sc in scenarios:
    for act in db:
        temp = act['name']
        # Check if the flow is valid and add to the flow list
        if ("pill" in temp or( "vial" in temp and "sc" in temp) or 'combined' in temp) and sc in temp:
            flow.append(act)

flow

['pill sc1' (unit, DK, None),
 'vial sc1' (kilogram, DK, None),
 'pill sc2' (unit, DK, None),
 'vial sc2' (kilogram, DK, None),
 'combined sc3' (kilogram, DK, None)]

In [6]:
dct = {f: {} for f in flow}
dct

{'pill sc1' (unit, DK, None): {},
 'vial sc1' (kilogram, DK, None): {},
 'pill sc2' (unit, DK, None): {},
 'vial sc2' (kilogram, DK, None): {},
 'combined sc3' (kilogram, DK, None): {}}

In [130]:
def find_exchanges(exc):
    for lvl in exc.input.exchanges():
        if "ev391cutoff" == lvl['database'] and "technosphere" in lvl["type"]:
            dct[act].update({lvl.input: exc['amount'] * lvl['amount']})
        else:
            find_exchanges(lvl)

In [ ]:
def find_exchanges(exc, dct, act):
    stack = [exc]
    while stack:
        current_exc = stack.pop()
        try:
            for lvl in current_exc.input.exchanges():
                # print(lvl["name"], lvl)
                if "ev391cutoff" == lvl['database'] and lvl["type"] == "technosphere":
                    dct[act].update({lvl.input: current_exc['amount'] * lvl['amount']})
                else:
                    if lvl.input != lvl.output:
                        stack.append(lvl)
        except KeyError as e:
            print(f"{e} for {lvl['name']}")

In [ ]:
def extract_exchanges_for_MC(flow):
    dct = {f: {} for f in flow}
    for act in flow:
        for exc in act.exchanges():
            if "technosphere" in exc["type"]:
                if "ev391cutoff" == exc['database']:
                    dct[act].update({exc.input: exc['amount']})
                else:
                    find_exchanges(exc, dct, act)
    return dct

In [3]:
path = r'C:/Users/ruw/Desktop'
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, rf'{path}\RA\penicilin\Libaries')

In [4]:
import reload_lib as rl

In [5]:
import MC
rl.reload_lib(MC)

In [6]:
from MC import monte_carlo as mc
from life_cycle_assessment import lcia_impact_method

In [8]:
rl.reload_lib(MC)
init = mc()
dct, uncert = init.extract_exchanges_for_MC()

activities = init.actvities
activities

['pill sc1' (unit, DK, None),
 'vial sc1' (kilogram, DK, None),
 'pill sc2' (unit, DK, None),
 'vial sc2' (kilogram, DK, None),
 'combined sc3' (kilogram, DK, None)]

In [ ]:
import bw2calc as bc
import bw2data as bd
import brightway2 as bw

In [109]:
method = lcia_impact_method()[1]
method

('ReCiPe 2016 v1.03, midpoint (H) - no biogenic',
 'climate change',
 'global warming potential (GWP1000)')

In [13]:
bd.projects.set_current("Penicillin")

In [14]:
import numpy as np

In [ ]:
uncert_dct = {}
techno = {}
for key, dct_act in init.dct.items():
    uncert_dct[key] = {}
    techno[key] = {}
    for act in dct_act.keys():
        for exc in act.exchanges():
            unc = exc.uncertainty
            if exc["type"] == "technosphere"  and unc["uncertainty type"] == 2:
                techno[key].update({act : {exc.input : exc.uncertainty}})

In [ ]:
sc_uncrt = {}
for sc in activities:
    loc = 0
    scale = 0
    sc_uncrt[sc] = {}
    try:
        for act, uncrt_dct in techno[sc].items():
            for uncrt in uncrt_dct.values():
                loc += uncrt['loc']
                scale += uncrt['scale']
        sc_uncrt[sc].update({'loc' : loc/len(techno[sc])})
        sc_uncrt[sc].update({'scale' : scale/len(techno[sc])})
    except KeyError:
        pass

In [76]:
import pandas as pd
from time import time

In [93]:
sampled_data = {}

itterations = 2

df_mc = pd.DataFrame(0, index=activities, columns=list(range(1,itterations+1)), dtype=object)

# try:
results_dct = {}
for itt in range(itterations):
    start = time()
    func_unit_MC = []
    print(f"Iterration {itt+1} of {itterations}")
    for act, uncert in sc_uncrt.items():
        mean = np.exp(uncert['loc'])
        sigma = uncert['scale']
        new_val = np.random.lognormal(mean, sigma)
        func_unit_MC.append({act : new_val})

    bd.calculation_setups[f'MC_{itt+1}'] = {'inv': func_unit_MC, 'ia': [method[0]]}
    MC_res = bc.MultiLCA(f'MC_{itt+1}')
    MC_results_array = MC_res.results

    # Store results in DataFrame
    for idx, res in enumerate(MC_results_array):
        df_mc.iat[idx, itt] = res
    print(f"Iterration {itt+1} took {round(time() - start,2)} seconds")

df_mc

Iterration 1 of 2
Iterration 1 took 77.69 seconds
Iterration 2 of 2
Iterration 2 took 52.94 seconds


,1,2
"[production amount, reference product, location, unit, name, worksheet name, database, code]",9.203258673846099e-05,0.00017071613016885867
"[production amount, reference product, location, unit, name, worksheet name, database, code]",0.004141890951158957,0.002189797496091104
"[production amount, reference product, location, unit, name, worksheet name, database, code]",0.0026473453872402908,0.003321894417903362
"[production amount, reference product, location, unit, name, worksheet name, database, code]",0.03444703191899663,0.04016318501092259
"[production amount, reference product, location, unit, name, worksheet name, database, code]",0.0657685167297627,0.015754121734646673


In [100]:
dct_MC = {}

for idx,row in df_mc.iterrows():
    arr_temp = np.zeros(itterations)
    for col in df_mc.columns:
        arr_temp[col-1] = row[col]
        # print(arr_res[col-1], idx,row[col])
    dct_MC[idx] = arr_temp
dct_MC

{'pill sc1' (unit, DK, None): array([9.20325867e-05, 1.70716130e-04]),
 'vial sc1' (kilogram, DK, None): array([0.00414189, 0.0021898 ]),
 'pill sc2' (unit, DK, None): array([0.00264735, 0.00332189]),
 'vial sc2' (kilogram, DK, None): array([0.03444703, 0.04016319]),
 'combined sc3' (kilogram, DK, None): array([0.06576852, 0.01575412])}

In [106]:
data_proccessing = ['Mean', 'Median', 'Standard Deviation', 'Minimum', 'Maximum']
data_proccessing_dct = {}

for act, arr in dct_MC.items():
    data_proccessing_dct[act] = {}
    for dp in data_proccessing:
        print(act, arr, dp)
        if 'mean' in dp.lower():
            data_proccessing_dct[act].update({dp : np.mean(arr)})
                # print(index)
        elif 'median' in dp.lower():
            data_proccessing_dct[act].update({dp : np.median(arr)})
            # print(index)
        elif 'standard' in dp.lower():
            data_proccessing_dct[act].update({dp : np.std(arr)})
            # print(index)
        elif 'minimum' in dp.lower():
            data_proccessing_dct[act].update({dp : np.min(arr)})
            # print(index)
        elif 'maximum' in dp.lower():
            data_proccessing_dct[act].update({dp : np.max(arr)})
data_proccessing_dct

'pill sc1' (unit, DK, None) [9.20325867e-05 1.70716130e-04] Mean
'pill sc1' (unit, DK, None) [9.20325867e-05 1.70716130e-04] Median
'pill sc1' (unit, DK, None) [9.20325867e-05 1.70716130e-04] Standard Deviation
'pill sc1' (unit, DK, None) [9.20325867e-05 1.70716130e-04] Minimum
'pill sc1' (unit, DK, None) [9.20325867e-05 1.70716130e-04] Maximum
'vial sc1' (kilogram, DK, None) [0.00414189 0.0021898 ] Mean
'vial sc1' (kilogram, DK, None) [0.00414189 0.0021898 ] Median
'vial sc1' (kilogram, DK, None) [0.00414189 0.0021898 ] Standard Deviation
'vial sc1' (kilogram, DK, None) [0.00414189 0.0021898 ] Minimum
'vial sc1' (kilogram, DK, None) [0.00414189 0.0021898 ] Maximum
'pill sc2' (unit, DK, None) [0.00264735 0.00332189] Mean
'pill sc2' (unit, DK, None) [0.00264735 0.00332189] Median
'pill sc2' (unit, DK, None) [0.00264735 0.00332189] Standard Deviation
'pill sc2' (unit, DK, None) [0.00264735 0.00332189] Minimum
'pill sc2' (unit, DK, None) [0.00264735 0.00332189] Maximum
'vial sc2' (kilogra

{'pill sc1' (unit, DK, None): {'Mean': 0.00013137435845365984,
  'Median': 0.00013137435845365984,
  'Standard Deviation': 3.934177171519884e-05,
  'Minimum': 9.203258673846099e-05,
  'Maximum': 0.00017071613016885867},
 'vial sc1' (kilogram, DK, None): {'Mean': 0.00316584422362503,
  'Median': 0.00316584422362503,
  'Standard Deviation': 0.0009760467275339265,
  'Minimum': 0.002189797496091104,
  'Maximum': 0.004141890951158957},
 'pill sc2' (unit, DK, None): {'Mean': 0.0029846199025718263,
  'Median': 0.0029846199025718263,
  'Standard Deviation': 0.0003372745153315355,
  'Minimum': 0.0026473453872402908,
  'Maximum': 0.003321894417903362},
 'vial sc2' (kilogram, DK, None): {'Mean': 0.037305108464959616,
  'Median': 0.037305108464959616,
  'Standard Deviation': 0.0028580765459629807,
  'Minimum': 0.03444703191899663,
  'Maximum': 0.04016318501092259},
 'combined sc3' (kilogram, DK, None): {'Mean': 0.04076131923220468,
  'Median': 0.04076131923220468,
  'Standard Deviation': 0.0250071